<a href="https://colab.research.google.com/github/ggwnsghgg/book_project/blob/main/2021%20/%2004%20/%2003%20/%20%EC%9C%A0%EC%82%AC%EB%8F%84%20%EA%B2%80%EC%A6%9D_fin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 필요한 패키지 install

In [1]:
!pip install pymysql


# 형태소 기반 토크나이징 (Konlpy)
!python3 -m pip install konlpy
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)



#한국어 문장 분리모델
!pip install kss


!sudo apt-get install curl git
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)


#mecab 코랩서 오류 안나게!
!apt-get update
!apt-get install g++ openjdk-8-jdk 
!pip3 install konlpy JPype1-py3
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

     |████████████████████████████████| 51kB 5.7MB/s 
     |████████████████████████████████| 19.4MB 1.2MB/s 
     |████████████████████████████████| 460kB 45.6MB/s 
     |████████████████████████████████| 92kB 10.5MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
Installing automake (A dependency for mecab-ko)
Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:6 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bio

In [3]:
import os
import kss
import re
import pandas as pd
import numpy as np
import konlpy
from tqdm import tqdm
from konlpy.tag import Mecab
import pymysql

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from google.colab import drive
drive.mount('/content/drive')



# # mysql용 데이터 가져오기
# db = pymysql.connect(host='220.86.191.27', port=3306, user='playdata', password='playdata123!', db='bookdata', charset='utf8')

# curs = db.cursor(pymysql.cursors.DictCursor)  

# query = '''
# select *
#     from books_data'''
# df_books_data = pd.read_sql(query, db)

Mounted at /content/drive


In [4]:
# 구글 드라이브

path = '/content/drive/MyDrive/'
df_books = pd.read_excel(os.path.join(path, 'book_fin.xlsx'))

In [5]:
df_books

,Unnamed: 0,index,book_id,title,genre,words,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,0,0,0,요리코를 위해,추리/미스터리,리코 수 이자 위 누구 출발 탐정 사체 심판 무대 남자 진실 딸 패배 세팅 파악 괴...,NaN,NaN,1.0,NaN,NaN,NaN
1,1,1,1,그녀는 다 계획이 있다,추리/미스터리,미래 복고 동료 교코 호텔 원대 미스터리 직장 컴 자신 계획 터치 날 감사 파티 살...,NaN,NaN,1.0,NaN,NaN,NaN
2,2,2,2,블랙 쇼맨과 이름 없는 마을의 살인,추리/미스터리,소식 통화 살 순간 너 결혼식 시작 취직 통 약혼자 전화 경 생전 결혼 마지막 고향...,NaN,NaN,1.0,NaN,NaN,NaN
3,3,3,3,돌이킬 수 없는 약속,추리/미스터리,소설 수 야쿠 가쿠 전 작가 미스터리 죄 포상 놈 딸 제 란 마루 숨 추리 약속 주...,NaN,NaN,1.0,NaN,NaN,NaN
4,4,4,4,루팡의 딸,추리/미스터리,이 발생 카즈마 사체 자신 딸 코 날 대부 변 경찰 손녀 오 괴한 신분 도둑 발견 ...,NaN,NaN,1.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
11154,11154,10498,65525,미주부동산 2,로맨스,사람 곤란 때 몸 품 눈물 나 생각 자신 감득 남자 피부 탓 것 어른 내 제,NaN,NaN,NaN,NaN,NaN,NaN
11155,11155,10499,65526,미주부동산 1,로맨스,아이스커피 키스 듯 못 세상 결여 콜라 자동차 김 차지 마이크 노래방 하느님 희극 ...,NaN,NaN,NaN,NaN,NaN,NaN
11156,11156,10500,65528,나쁜 결혼,로맨스,존경 사랑 남편 감정 태경 마음 속 여자 의무 남자 지우 수 눈길 아내 말 결혼 당신,NaN,NaN,NaN,NaN,NaN,NaN
11157,11157,10501,45307,소오강호 1,무협,세상은 정의를 외치지만 추악하다. 권력을 움켜쥐려는 자들은 불의도 서슴지 않는다. ...,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df_books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11159 entries, 0 to 11158
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   11159 non-null  int64  
 1   index        11159 non-null  int64  
 2   book_id      11159 non-null  int64  
 3   title        11159 non-null  object 
 4   genre        11159 non-null  object 
 5   words        11159 non-null  object 
 6   Unnamed: 6   0 non-null      float64
 7   Unnamed: 7   0 non-null      float64
 8   Unnamed: 8   2489 non-null   float64
 9   Unnamed: 9   0 non-null      float64
 10  Unnamed: 10  0 non-null      float64
 11  Unnamed: 11  1 non-null      object 
dtypes: float64(5), int64(3), object(4)
memory usage: 1.0+ MB


# 유사도 검증 

In [23]:
# 유사도 검증에 필요한 패키지 import
import pandas as pd
from numpy import dot
from numpy.linalg import norm
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity



# words(스토리)에 대해서 tf_idf 수행
tfidf = TfidfVectorizer()
tfidf_matrix_mys = tfidf.fit_transform(df_books['words'])
print(tfidf_matrix_mys.shape)

# 유사도 점수화
cosine_matrix_mys = cosine_similarity(tfidf_matrix_mys, tfidf_matrix_mys)

# 책 타이틀 
books_mys_id = {}
for i, c in enumerate(df_books['title']): books_mys_id[i] = c
        
# 책 id
id2books_mys = {}
for i, c in books_mys_id.items(): id2books_mys[c] = i


def get_recommendations(title, cosine_sim=cosine_matrix_mys):

    idx = id2books_mys[title]
    sim_scores_mys = [(i,c) for i, c in enumerate(cosine_matrix_mys[idx]) if i !=idx] 
    sim_scores_mys = sorted(sim_scores_mys, key =lambda x: x[1], reverse = True)
    sim_scores_mys[0:10]
    

    sim_scores_mys = [(books_mys_id[i], score) for i, score in sim_scores_mys[0:10]]
    return sim_scores_mys

(11159, 20420)


In [26]:
# 유사도 검증하고 하고자하는 책 입력
get_recommendations('하늘을 나는 말')

[('메르카토르는 이렇게 말했다', 0.2901931483429056),
 ('벚꽃 지는 계절에 그대를 그리워하네', 0.25641629315941594),
 ('귀족탐정', 0.24936238465048238),
 ('시소 게임', 0.2472706271806308),
 ('리라장 사건', 0.23818521651676033),
 ('고양이는 알고 있다', 0.2380710710138387),
 ('나가에의 심야상담소', 0.22875141995761955),
 ('경관의 피', 0.22820435442773626),
 ('잘린 머리처럼 불길한 것', 0.2260263078673),
 ('지나가는 녹색 바람', 0.22365816217219048)]

(2489, 9006)
